In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msng    # normally this is imported under the alias 'msno'
                            # but this is the name of the user id column

In [ ]:
import datetime as dt

In [ ]:
import sys  
sys.path.append('/Users/kendra/data_science/Projects/sharedCode')

import util as u
import assess_clf_models as acm

%load_ext autoreload
%autoreload 2

In [ ]:
# X = u.open_pkl('Data/X_train.pkl')
df = u.open_pkl('Data/user_df.pkl')

In [ ]:
df.columns

In [ ]:
# added these to previous data processing script after first time through:
if 1 == 0:
    df.rename(columns={'total_secs_count':'num_log_entries'}, inplace=True)

    df['time_since_registration'] = round((dt.datetime(year=2017, month=4, day=30) - df['registration_init_time']) 
                                     / dt.timedelta(days=30), 2)

In [ ]:
df.info()

In [ ]:
df['registered_via'] = df['registered_via'].astype('category')

In [ ]:
sns.distplot(df['time_since_registration'].dropna())

### Missing Data

In [ ]:
msng.matrix(df)

In [ ]:
msng.bar(df)

In [ ]:
msng.heatmap(df)

In [ ]:
len(df.dropna())/len(df)

In [ ]:
len(df['trans_count'].dropna())/len(df)

#### Missing data:
- At the least, need to keep transaction data
- At the worst, drop all na's and retain 75% of data
**For now: drop all na's**

In [ ]:
df_drop = df.dropna()
df_drop.reset_index(drop=True, inplace=True)

#### Correlated features

In [ ]:
corr = df_drop.corr()

fig = plt.figure(figsize=(12,10))
sns.heatmap(corr, vmin=-1, vmax=1);

- num_100, num_unq, total num songs (mean & sum), total_secs sum
    - compute PCA
- short_pct & full_pct (inverse)
    - keep short_pct
    - drop num_25 thru num_985

### PCA - song features

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
pca_songs = PCA(n_components=1)
df_songs_pca = df_drop[['num_100_mean', 'num_unq_mean', 'num_songs_mean', 'num_songs_sum',
       'total_secs_mean', 'total_secs_sum']]
songs_pca = pca_songs.fit_transform(df_songs_pca)

In [ ]:
sns.distplot(songs_pca)

### Transaction features

In [ ]:
trans_cols = ['payment_plan_days_mode', 'payment_plan_days_sum',
       'plan_list_price_mode', 'plan_list_price_sum',
       'actual_amount_paid_mode', 'actual_amount_paid_sum',
        'plan_actual_diff_abs_max', 'plan_actual_diff_mode',
       'is_auto_renew_mode', 'is_auto_renew_sum', 'is_cancel_mode',
       'is_cancel_sum', 'trans_count']

In [ ]:
sns.heatmap(corr.loc[trans_cols, trans_cols], vmin=-1, vmax=1);

- transaction features:
    - PCA on payment_plan_days mode or sum, plan_list_price sum or mode or actual sum or mode
    - one of plan_actual_diff abs_max of mode
    - keep is_auto_renew mode, drop sum
    - keep is_cancel_mode, drop sum
    - keep trans_count

In [ ]:
pca_trans = PCA(n_components=1)
df_trans_pca = df_drop[['payment_plan_days_mode', 'payment_plan_days_sum',
       'plan_list_price_mode', 'plan_list_price_sum',
       'actual_amount_paid_mode', 'actual_amount_paid_sum']]
trans_pca = pca_songs.fit_transform(df_trans_pca)

In [ ]:
sns.distplot(trans_pca)

## Feature Set 1

In [ ]:
fs_cols_1 = ['registered_via', 
             'short_pct_mean', 
             'payment_method_most_common_mode',
             'plan_actual_diff_abs_max',
             'is_auto_renew_mode', 'is_cancel_mode',
             'trans_count', 
             'time_since_registration']

df_feat1 = df_drop[fs_cols_1]
df_feat1 = df_feat1.join(pd.DataFrame(songs_pca, columns=['song_pca']))
df_feat1 = df_feat1.join(pd.DataFrame(trans_pca, columns=['transactions_pca']))

df_feat1.head()

In [ ]:
df_feat1.info()

In [ ]:
fs_cols_2 = fs_cols_1 + ['num_songs_mean', 'actual_amount_paid_mode']
df_feat2 = df_drop[fs_cols_2]

df_feat2.head()

### Pipeline / Pre-processing
- normalize numerical categories
- encode categorial
- leave Boolean (1/0) as-is

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, FunctionTransformer
from sklearn.pipeline import Pipeline, FeatureUnion

#### Numerical

In [271]:
# numerical features are in same positions in feature set 1 & 2, so only need to define one list & preprocessor
feat_num = ['short_pct_mean', 
             'plan_actual_diff_abs_max', 
             'trans_count', 
             'time_since_registration', 
             'song_pca', # num_songs_mean in feature set 2
             'transactions_pca'] # 'actual_amount_paid_mode' in feature set 2

# define this list for feature names
feat2_num = ['short_pct_mean', 
             'plan_actual_diff_abs_max', 
             'trans_count', 
             'time_since_registration', 
             'num_songs_mean',
             'actual_amount_paid_mode']

feat_num_idx = [list(df_feat1.columns).index(x) for x in feat_num]

# define Scaling preprocessor
preproc_scale = ColumnTransformer(transformers=[
    ('num', StandardScaler(), feat_num_idx)
])

# for models that don't require scaling, we want to pass-through these features:
preproc_num_pass = ColumnTransformer(transformers=[
    ('num', 'passthrough', feat_num_idx)
])

# feature names = feat_num or feat2_num

#### Categorial

In [272]:
import re

def fix_ohe_names(x):
    s = re.sub('x0', 'reg_via', x)
    s = re.sub('.0', '', s)
    return s

cat_cols = ['registered_via']
cat_cols_idx = [list(df_feat1.columns).index(x) for x in cat_cols]

preproc_ohe = ColumnTransformer(transformers=[('cat', OneHotEncoder(categories='auto'), cat_cols_idx)])

# fit to get feature names
preproc_ohe.fit(df_feat1)
feat_ohe = preproc_ohe.named_transformers_['cat'].get_feature_names()
feat_ohe = feat_ohe.tolist()

feat_ohe = list(map(fix_ohe_names, feat_ohe))
feat_ohe

['reg_via_3', 'reg_via_4', 'reg_via_7', 'reg_via_9', 'reg_via_13']

#### Boolean

In [274]:
pass_cols = ['payment_method_most_common_mode', 'is_auto_renew_mode', 'is_cancel_mode']
pass_cols_idx = [list(df_feat1.columns).index(x) for x in pass_cols]

preproc_pass = ColumnTransformer(transformers=[
        ('as_is', 'passthrough', pass_cols_idx), 
])

# feature names = pass_cols

### Sewing them together
**Scaling**

In [275]:
pipe_with_scale = Pipeline([
    ('all', FeatureUnion([
        ('ohe', preproc_ohe),
        ('num', preproc_scale),
        ('pass', preproc_pass)
    ])
    )
])

feat_names1 = feat_ohe + feat_num + pass_cols
feat_names2 = feat_ohe + feat2_num + pass_cols

**Scaling not needed**

In [277]:
pipe_no_scale = Pipeline([
    ('all', FeatureUnion([
        ('ohe', preproc_ohe),
        ('num', preproc_num_pass),
        ('pass', preproc_pass)
    ])
    )
])

Notes:
- CatBoost will not require any pre-processing. Will simply need to feed in the categorical columns (index 0)
- can also try feeding in data without pre-processing into tree-based models
    - see: https://towardsdatascience.com/one-hot-encoding-is-making-your-tree-based-ensembles-worse-heres-why-d64b282b5769

#### Test

In [278]:
trX1_scale = pipe_with_scale.fit_transform(df_feat1)
test_data = trX1_scale.todense()[0].tolist()[0]

In [279]:
trX1 = pipe_no_scale.fit_transform(df_feat1)
test_data_no_scale = trX1.todense()[0].tolist()[0]

In [282]:
test_df = pd.DataFrame({'not scaled':test_data_no_scale, 
             'scaled':test_data},
            index=feat_names1)

test_df.join(df_feat1.loc[0])

,not scaled,scaled,0
reg_via_3,1.000000,1.000000,NaN
reg_via_4,0.000000,0.000000,NaN
reg_via_7,0.000000,0.000000,NaN
reg_via_9,0.000000,0.000000,NaN
reg_via_13,0.000000,0.000000,NaN
short_pct_mean,0.000000,-1.282839,0
plan_actual_diff_abs_max,0.000000,-0.036292,0
trans_count,1.000000,-0.228602,1
time_since_registration,40.800000,-0.122216,40.8
song_pca,-134821.246306,-0.731279,-134821


## Modeling
Competition submissions were evaluated by log-loss, so include it in our metrics

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier, BaggingClassifier
from catboost import CatBoostClassifier

In [ ]:
log_reg = LogisticRegression(solver='liblinear')
rand_for = RandomForestClassifier(n_estimators=10)
grad_boost = GradientBoostingClassifier()
cb = CatBoostClassifier(cat_features=cat_cols_idx, logging_level='Silent')
dtb = BaggingClassifier(base_estimator=DecisionTreeClassifier())

In [ ]:
y = df_drop['is_churn']

In [ ]:
y.value_counts(1)

Be aware our dataset is imbalanced

In [283]:
fit_feat1_log_reg = acm.assess_model_2(pipe_with_scale, log_reg, df_feat1, y)

In [284]:
fit_feat2_log_reg = acm.assess_model_2(pipe_with_scale, log_reg, df_feat2, y)

In [285]:
fit_feat1_log_reg

Precision-0                    0.957912
Recall-0 (Specificty)          0.993849
F1score-0                      0.975550
Precision-1                    0.802278
Recall-1 (Sensitivity)         0.363668
F1score-1                      0.500460
TN                        134986.200000
FN                          5931.000000
FP                           835.400000
TP                          3389.600000
AUC                            0.920155
Accuracy                       0.953381
LogLoss                        1.610172
dtype: float64

In [286]:
fit_feat2_log_reg

Precision-0                    0.957619
Recall-0 (Specificty)          0.993596
F1score-0                      0.975276
Precision-1                    0.793785
Recall-1 (Sensitivity)         0.359204
F1score-1                      0.494578
TN                        134951.800000
FN                          5972.600000
FP                           869.800000
TP                          3348.000000
AUC                            0.922288
Accuracy                       0.952857
LogLoss                        1.628257
dtype: float64

The two feature sets (1 = with PCA, 2 = selected un-transformed features) perform similarly in a logistic regression model

In [287]:
mini_results_rf = acm.assess_model_2(pipe_no_scale, rand_for, df_feat1, y)
mini_results_gb = acm.assess_model_2(pipe_no_scale, grad_boost, df_feat1, y)
mini_results_dtb = acm.assess_model_2(pipe_no_scale, dtb, df_feat1, y)

In [288]:
mini_results_rf

Precision-0                    0.971632
Recall-0 (Specificty)          0.992615
F1score-0                      0.982012
Precision-1                    0.842953
Recall-1 (Sensitivity)         0.577688
F1score-1                      0.685538
TN                        134818.600000
FN                          3936.200000
FP                          1003.000000
TP                          5384.400000
AUC                            0.895119
Accuracy                       0.965970
LogLoss                        1.175363
dtype: float64

In [289]:
mini_results_gb

Precision-0                    0.971828
Recall-0 (Specificty)          0.993936
F1score-0                      0.982758
Precision-1                    0.867863
Recall-1 (Sensitivity)         0.580135
F1score-1                      0.695388
TN                        134998.000000
FN                          3913.400000
FP                           823.600000
TP                          5407.200000
AUC                            0.949705
Accuracy                       0.967363
LogLoss                        1.127245
dtype: float64

In [290]:
mini_results_dtb

Precision-0                    0.971662
Recall-0 (Specificty)          0.992546
F1score-0                      0.981993
Precision-1                    0.841843
Recall-1 (Sensitivity)         0.578182
F1score-1                      0.685524
TN                        134809.200000
FN                          3931.600000
FP                          1012.400000
TP                          5389.000000
AUC                            0.896821
Accuracy                       0.965937
LogLoss                        1.176505
dtype: float64

In [292]:
mini_results_cb = acm.assess_model_only(cb, df_feat1, y)

In [293]:
comp_df = pd.DataFrame([fit_feat1_log_reg, fit_feat2_log_reg, mini_results_rf, mini_results_gb, mini_results_dtb, 
                        mini_results_cb],
             index=['LogReg','LogReg-FS2','RandFor','GradBst','Bagged-DecTr', 'Cat Boost'])
comp_df

,Precision-0,Recall-0 (Specificty),F1score-0,Precision-1,Recall-1 (Sensitivity),F1score-1,TN,FN,FP,TP,AUC,Accuracy,LogLoss
LogReg,0.957912,0.993849,0.975550,0.802278,0.363668,0.500460,134986.2,5931.0,835.4,3389.6,0.920155,0.953381,1.610172
LogReg-FS2,0.957619,0.993596,0.975276,0.793785,0.359204,0.494578,134951.8,5972.6,869.8,3348.0,0.922288,0.952857,1.628257
RandFor,0.971632,0.992615,0.982012,0.842953,0.577688,0.685538,134818.6,3936.2,1003.0,5384.4,0.895119,0.965970,1.175363
GradBst,0.971828,0.993936,0.982758,0.867863,0.580135,0.695388,134998.0,3913.4,823.6,5407.2,0.949705,0.967363,1.127245
Bagged-DecTr,0.971662,0.992546,0.981993,0.841843,0.578182,0.685524,134809.2,3931.6,1012.4,5389.0,0.896821,0.965937,1.176505
Cat Boost,0.972699,0.994085,0.983276,0.873170,0.593417,0.706603,135018.2,3789.6,803.4,5531.0,0.954258,0.968355,1.092978
